In [4]:
import alpaca_backtrader_api
import backtrader as bt
from datetime import datetime

In [9]:
ALPACA_API_KEY ="PK8SSSXD3FX164IYREJ5"
ALPACA_SECRET_KEY ="NPKEgtQyt3fsRodbOqn4d46V19XI5W9S8Cv64dDT"
ALPACA_PAPER = True

In [34]:
from alpaca_trade_api.rest import REST, TimeFrame
api = REST(ALPACA_API_KEY,
           ALPACA_SECRET_KEY,
           'https://paper-api.alpaca.amrkets'
          )

data = api.get_bars("AAPL", TimeFrame.Day, "2019-11-15", "2021-11-15", adjustment='raw').df
# new_data = pd.DataFrame(index=range(0,len(data)),columns=['timestamp', 'close'])
# new_data

In [53]:
symbol = 'AAPL'
bar = api.get_barset(symbol, 'day', limit=730)

c = []
t = []
idx = []

for i in range(len(bar[symbol])):
    temp_c = c.append(bar[symbol][i].c) 
    temp_t = t.append(bar[symbol][i].t)
    temp_idx = idx.append(i)
    i = i + 1

df_bar_t = pd.DataFrame(t, idx,columns = ['Datetime'])
df_bar_c = pd.DataFrame(c, idx, columns = ['Close'])

new_data = pd.merge(df_bar_t,df_bar_c, left_index = True, right_index = True)
new_data



,Datetime,Close
0,2019-01-08 00:00:00-05:00,150.6300
1,2019-01-09 00:00:00-05:00,153.3700
2,2019-01-10 00:00:00-05:00,153.5500
3,2019-01-11 00:00:00-05:00,152.0534
4,2019-01-14 00:00:00-05:00,149.7600
...,...,...
725,2021-11-22 00:00:00-05:00,161.0800
726,2021-11-23 00:00:00-05:00,161.4200
727,2021-11-24 00:00:00-05:00,161.9400
728,2021-11-26 00:00:00-05:00,156.8800


In [3]:
#import packages
import pandas as pd
import numpy as np
 
#to plot within notebook
import matplotlib.pyplot as plt
 
#setting figure size
from matplotlib.pylab import rcParams

In [56]:
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
 
#creating dataframe
symbol = 'AAPL'
bar = api.get_barset(symbol, 'day', limit=730)

c = []
t = []
idx = []

for i in range(len(bar[symbol])):
    temp_c = c.append(bar[symbol][i].c) 
    temp_t = t.append(bar[symbol][i].t)
    temp_idx = idx.append(i)
    i = i + 1

df_bar_t = pd.DataFrame(t, idx,columns = ['Datetime'])
df_bar_c = pd.DataFrame(c, idx, columns = ['Close'])

new_data = pd.merge(df_bar_t,df_bar_c, left_index = True, right_index = True)
 
#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)
 
#creating train and test sets
dataset = new_data.values
 
train = dataset[0:511,:]
valid = dataset[511:,:]
 
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

#--------------------------

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)
 
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
 
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))
 
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)
 
#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)
 
X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)
 
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

ModuleNotFoundError: No module named 'keras'

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms